# Deepface Face Recognition test 

### Import relevant libraries:

In [190]:
# !pip install deepface
from deepface import DeepFace

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re

# change plot output to white
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)

result_df = pd.read_csv('./result/result_debface.csv')

In [191]:
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images
0,VGG-Face,LFW,88.0,100.0,88.0,9164


#### Paramater settings:

In [208]:
# select a DATASET from data folder:
# LFW, LFW-original
DATASET = "LFW"

# select a dataset from additional-data folder:
# tzuyu, kpop
DATASET_TEST = "tzuyu"

# select a model to run:
# models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
MODEL = "ArcFace"

#### Reset directory and test instances 

In [209]:
import os

# Test additional data directory:
# dir_name = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_name = f"./data/{DATASET}"
all_dir_names = os.listdir(dir_name)

# Removes previously stored representations_FACE.pkl, as adding new instances requires to re-test the model
for item in all_dir_names:
    if item.endswith(".pkl"):
        os.remove(os.path.join(dir_name, item))

### Function to perform test with Deepface

In [180]:
def deepface_get_accuracy(img_dir_path, model_used):

    # store the results of each verification:
    tp, tn, fp, fn = 0, 0, 0, 0
    image_files = os.listdir(img_dir_path)
    truth_images = []    
    false_images = []    
    
    # If the directory has less than 2 images there is no need to test face recognition
    if len(image_files) < 2:
        return None

    else:
        # select the first image 
        image_1 = f"{img_dir_path}/{image_files[0]}"

        # loop through the other images and verify if they are the same
        for img in image_files[1:]:
            
            try:
                match = re.search(f"_\d.*\.jpg", str(img))
                # Find all annotation of the files, replace <NAME> with the desired person to test
                # get image name to replace the match string with empty string
                # NAME_\d\.jpg
                # NAME_\d.*\.jpg
                NAME = img.replace(match.group(0), "")
            except AttributeError:
                print("IMAGE: " + image_1)
                exit(1)

            
            flag = None
            if re.search(f"{NAME}_\d.*\.jpg", str(img)):
                truth_images.append(img)
                flag = True
            else:
                false_images.append(img)    
                flag = False

            try:
                img_path = f"{img_dir_path}/{img}"
                verification = DeepFace.verify(img1_path = image_1, img2_path = img_path, model_name=model_used, enforce_detection=False)
                verification_res = verification["verified"]
                
                # Truth Positive if flag and predicted are True:
                if flag and verification_res:
                    tp += 1
                # Truth Negative if flag and predicted are False:
                elif flag == False and verification_res == False:
                    tn += 1

                # False Positive if flag is False and predicted is True:
                elif flag == False and verification_res == True:
                    fp += 1

                # False Negative if flag is True and predicted is False:
                elif flag == True and verification_res == False:
                    fn += 1        

            except Exception as e:
                print(e)

        # confusion matrix
        cm_acc = round((tp + tn) / (tp + tn + fp + fn), 2) * 100

        return {"cm_acc": cm_acc, "tp": tp, "tn":tn, "fp":fp, "fn":fn, "image_files": len(image_files)}


### Dataset Checkup:

In [181]:
def deepface_run_dataset(dataset, dataset_path, MODEL):

    avg_truth_pos = 0
    avg_truth_neg = 0
    avg_false_pos = 0
    avg_false_neg = 0
    total_imgs = 0
    
    people = os.listdir(dataset_path)
    for plp in people:
        img_path = f"{dataset_path}/{plp}"
        data_res = deepface_get_accuracy(img_path, MODEL)

        if data_res != None:
            avg_truth_pos += data_res["tp"]
            avg_truth_neg += data_res["tn"]
            avg_false_pos += data_res["fp"]
            avg_false_neg += data_res["fn"]
            total_imgs += data_res["image_files"]

    # calculate confusion matrix:
    # accuracy:
    cm_acc = round((avg_truth_pos + avg_truth_neg) / (avg_truth_pos + avg_truth_neg + avg_false_pos + avg_false_neg), 2) * 100
    # precision:
    cm_pre = round( (avg_truth_pos) / (avg_truth_pos + avg_false_pos), 2) * 100
    # recall: 
    cm_rec = round( (avg_truth_pos) / (avg_truth_pos + avg_false_neg), 2) * 100


    return {"Model": MODEL, "Dataset":dataset, "CM_ACC": cm_acc, "Precision":cm_pre, "Recall":cm_rec,"Total Images": total_imgs},\
         {"TP": avg_truth_pos, "TN":avg_truth_neg, "FP":avg_false_pos, "FN":avg_false_neg}

### Execution of the Deepface test:

In [133]:
# deepface_res = deepface_get_accuracy(dir_name, MODEL)

1/1 [==============================] - 0s 120ms/step


In [210]:
deepface_res_dataset, cm_attr = deepface_run_dataset(DATASET, dir_name, MODEL)

arcface_weights.h5  will be downloaded to  C:\Users\EzLaser/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: C:\Users\EzLaser\.deepface\weights\arcface_weights.h5
100%|██████████| 137M/137M [00:16<00:00, 8.20MB/s] 


1/1 [==============================] - 0s 77ms/step


## Result of the Deepface test:

In [211]:
# confusion matrix accuracy:
# cm_acc = deepface_res["cm_acc"]
# truth_pos = deepface_res["tp"]
# truth_neg = deepface_res["tn"]
# false_pos = deepface_res["fp"]
# false_neg = deepface_res["fn"]                            

cm_acc = deepface_res_dataset["CM_ACC"]
cm_pre = deepface_res_dataset["Precision"]
cm_rec = deepface_res_dataset["Recall"]
truth_pos = cm_attr["TP"]
truth_neg = cm_attr["TN"]
false_pos = cm_attr["FP"]
false_neg = cm_attr["FN"]

# total images
# total_imgs = deepface_res["image_files"]
total_imgs = deepface_res_dataset["Total Images"]

print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
Truth Positive: {truth_pos} \n\
Truth Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================")

Accuracy of ArcFace is 83.0% out of 9164 images from LFW dataset.
The Precision is 100.0 and Recall is 83.0
Truth Positive: 6213 
Truth Negative: 0 
False Positive: 0 
False Negative: 1271 


In [212]:
result_df_new = pd.DataFrame([deepface_res_dataset])

frames = [result_df, result_df_new]
result_df = pd.concat(frames)

result_df = result_df.reset_index()
result_df.drop(columns=['index'], inplace=True)
result_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images
0,VGG-Face,LFW,88.0,100.0,88.0,9164
1,Facenet,LFW,65.0,100.0,65.0,9164
2,OpenFace,LFW,1.0,100.0,1.0,9164
3,DeepFace,LFW,26.0,100.0,26.0,9164
4,ArcFace,LFW,83.0,100.0,83.0,9164


In [197]:
# Write results to csv file
result_df.to_csv("./result/result_debface.csv", index=False)